In [14]:
install.packages("VancouvR")
install.packages("geojsonio")

library(geojsonio)
library(tidyverse)
library(dplyr)
library("knitr")

Warning message:
"package 'VancouvR' is in use and will not be installed"
Warning message:
"package 'geojsonio' is in use and will not be installed"


In [43]:
install.packages("cancensus")
library(cancensus)
options(cancensus.api_key='CensusMapper_f228791d9506a7a747ece66db73b95be')
options(cancensus.cache_path = 'C:\\Users\\Kaiyan Zhang\\Desktop\\econ326')

Warning message:
"package 'cancensus' is in use and will not be installed"


In [44]:
census_data <- get_census(dataset='CA16', 
regions=list(CMA="59933",CSD="5915022"), 
vectors=c("v_CA16_4951","v_CA16_5141"), 
labels="detailed", 
geo_format='sf', level='CT')|>
as.tibble()
# filter('`Region Name`' == 'Vancouver') # Installing Census data api but failed to select only Vancouver region

Reading vectors data from local cache.



Reading geo data from local cache.

Warning message in value[[3L]](cond):
"Unable to download recall database at this point."


In [45]:
colnames(census_data)

[1] "Shape Area"                                                                                                                                                                       
 [2] "Type"                                                                                                                                                                             
 [3] "Households"                                                                                                                                                                       
 [4] "Adjusted Population (previous Census)"                                                                                                                                            
 [5] "PR_UID"                                                                                                                                                                           
 [6] "Dwellings"                                                                                                                                                                        
 [7] "GeoUID"                                                                                                                                                                           
 [8] "Population"                                                                                                                                                                       
 [9] "CMA_UID"                                                                                                                                                                          
[10] "CSD_UID"                                                                                                                                                                          
[11] "CD_UID"                                                                                                                                                                           
[12] "Region Name"                                                                                                                                                                      
[13] "Area (sq km)"                                                                                                                                                                     
[14] "v_CA16_4951: Total - Income statistics in 2015 for the population aged 15 years and over in private households - 25% sample data"                                                 
[15] "v_CA16_5141: Total - Major field of study - Classification of Instructional Programs (CIP) 2016 for the population aged 15 years and over in private households - 25% sample data"
[16] "geometry"

In [46]:
distinct(census_data, 'Region Name')

"""Region Name"""
<chr>
Region Name


In [47]:
library(VancouvR)
search_cov_datasets("property-tax") %>%
  select(dataset_id,title)

dataset_id,title
<chr>,<chr>
property-tax-report-2011-2015,Property tax report 2011-2015
property-tax-report-2006-2010,Property tax report 2006-2010
property-tax-report-2016-2019,Property tax report 2016-2019
property-tax-report,Property tax report


In [48]:
tax_data <- get_cov_data(dataset_id = "property-tax-report-2016-2019",
                         where="tax_assessment_year='2016'",
                         select = "current_land_value, land_coordinate as tax_coord")|>
    as.tibble()
glimpse(tax_data) # read in the data of land value, and changed it coordname to tax coord, so it matched the property polygon data

Reading data from temporary cache



Warning message:
"There was 1 warning in `mutate()`.
ℹ In argument: `current_land_value =
  .Primitive("as.integer")(current_land_value)`.
Caused by warning:
! NAs introduced by coercion to integer range"


Rows: 199,924
Columns: 2
$ current_land_value <int> 126000, 1100000, 1028000, 825000, 922000, 878000, 1…
$ tax_coord          <chr> "58825549", "60125529", "61223603", "63025324", "59…


In [20]:
colnames(tax_data)

[1] "current_land_value" "tax_coord"

In [21]:
url = "https://raw.githubusercontent.com/Irene-Berezin/econ326/refs/heads/main/crimedata_csv_AllNeighbourhoods_AllYears/crimedata_csv_AllNeighbourhoods_AllYears.csv"
intentional_crime_data <- read_csv(url)|>
    as.tibble()|>
    select(TYPE, YEAR, NEIGHBOURHOOD)|>
    rename(type = TYPE, year = YEAR, neighbourhood = NEIGHBOURHOOD)|>
    filter(!type %in% c("Vehicle Collision or Pedestrian Struck (with Fatality)",
    "Vehicle Collision or Pedestrian Struck (with Injury)",
    "Mischief"))|>
    filter(year == 2016)|>
    group_by(type)|>
    arrange(year)

glimpse(intentional_crime_data)


Rows: 895823 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): TYPE, HUNDRED_BLOCK, NEIGHBOURHOOD
dbl (7): YEAR, MONTH, DAY, HOUR, MINUTE, X, Y

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Rows: 37,802
Columns: 3
Groups: type [8]
$ type          <chr> "Offence Against a Person", "Offence Against a Person", …
$ year          <dbl> 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016, 20…
$ neighbourhood <chr> "Kitsilano", "Kitsilano", "Kitsilano", "Kitsilano", "Kit…


In [22]:
local_area_boundary <- get_cov_data(dataset_id ="local-area-boundary")|>
    as_tibble()|>
    glimpse()

Rows: 22
Columns: 4
$ name         <chr> "Downtown", "Hastings-Sunrise", "Kerrisdale", "Marpole", …
$ geom         <chr> "{\"coordinates\": [[[-123.11226654052733, 49.29016494657…
$ geo_point_2d <chr> "49.28074707013743, -123.11656700827415", "49.27793405209…
$ geometry     <POLYGON [°]> POLYGON ((-123.1123 49.2901..., POLYGON ((-123.05…


In [49]:
glimpse(local_area_boundary)  # This dataframe stores the geom

Rows: 22
Columns: 4
$ name         <chr> "Downtown", "Hastings-Sunrise", "Kerrisdale", "Marpole", …
$ geom         <chr> "{\"coordinates\": [[[-123.11226654052733, 49.29016494657…
$ geo_point_2d <chr> "49.28074707013743, -123.11656700827415", "49.27793405209…
$ geometry     <POLYGON [°]> POLYGON ((-123.1123 49.2901..., POLYGON ((-123.05…


In [51]:
property_polygons <- get_cov_data(dataset_id="property-parcel-polygons")|>
    as_tibble()

Reading data from temporary cache



In [52]:
head(property_polygons)

civic_number,streetname,tax_coord,site_id,geom,geo_point_2d,geometry
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<POLYGON [°]>
3945,WELWYN ST,23970111,014527723,"{""coordinates"": [[[-123.07000584813854, 49.24987403564525], [-123.07000406207052, 49.24996166822032], [-123.07054814175194, 49.249969711822665], [-123.07055042153875, 49.249882087932896], [-123.07000584813854, 49.24987403564525]]], ""type"": ""Polygon""}","49.249921869270814, -123.0702771178116","POLYGON ((-123.07 49.24987,..."
3955,WELWYN ST,23970115,014527740,"{""coordinates"": [[[-123.0700076342004, 49.24978640306883], [-123.07000584813854, 49.24987403564525], [-123.07055042153875, 49.249882087932896], [-123.07055220732926, 49.24981422685563], [-123.07055271505841, 49.249794464033364], [-123.0700076342004, 49.24978640306883]]], ""type"": ""Polygon""}","49.24983424082434, -123.07027915667443","POLYGON ((-123.07 49.24979,..."
3969,WELWYN ST,23970119,014527782,"{""coordinates"": [[[-123.0700094202561, 49.249698770491044], [-123.0700076342004, 49.24978640306883], [-123.07055271505841, 49.249794464033364], [-123.07055500857015, 49.24970684013244], [-123.07051315332703, 49.249706218029374], [-123.0700094202561, 49.249698770491044]]], ""type"": ""Polygon""}","49.24974661188814, -123.07028119542241","POLYGON ((-123.07 49.2497, ..."
1399,E 19TH AV,69422893,007245190,"{""coordinates"": [[[-123.07625563593871, 49.253595208520274], [-123.07614707274392, 49.25359348107949], [-123.07613458693838, 49.25392767973197], [-123.07624315086562, 49.25392940718403], [-123.07625563593871, 49.253595208520274]]], ""type"": ""Polygon""}","49.25376144431683, -123.07619511161461",POLYGON ((-123.0763 49.2536...
1373,E 19TH AV,69422873,014613557,"{""coordinates"": [[[-123.07639374476234, 49.25359736608769], [-123.07638124686534, 49.25393155577966], [-123.07652004374253, 49.25393372173649], [-123.07653185361174, 49.25359953248523], [-123.07639374476234, 49.25359736608769]]], ""type"": ""Polygon""}","49.25376568232775, -123.0764567168287",POLYGON ((-123.0764 49.2536...
1351,E 19TH AV,69422851,010815431,"{""coordinates"": [[[-123.07653185361174, 49.25359953248523], [-123.07652004374253, 49.25393372173649], [-123.07665816726369, 49.25393587897811], [-123.07666997620092, 49.253601689713335], [-123.07653185361174, 49.25359953248523]]], ""type"": ""Polygon""}","49.2537677059162, -123.07659501019803",POLYGON ((-123.0765 49.2536...
